# Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain - Semantic search with metadata filtering with self-querying 

### Context
LLM question answering (Q+A) typically involves retrieval of documents relevant to the question followed by synthesis of the retrieved chunks into an answer by an LLM.  In practice, the retrieval step is necessary because the LLM context window is limited relative to the size of most text corpus of interest (e.g., LLM context windows range is limited to certain tokens). Anthropic recently released a Claude model with a 100k token context window.  With the advent of models with larger context windows, it is reasonable to wonder whether the document retrieval stage is necessary for many Q+A or chat use-cases.

One such retriever architectures with this retriever-less option is Semantic search with metadata filtering with self-querying option using Chroma database as vector database. 

### Pattern

Semantic with metadata filtering refers to the process of finding items that are both semantically relevant and meet specific filter conditions, such as the availability of a product in stock. In addition to the vector representation, the raw media format (e.g., document or image) contains a list of metadata, such as author and date.

To illustrate this, let's consider an example of image retrieval, specifically finding shoe images released after a certain date. Embedding the date directly into the vector representation is challenging, but we can overcome this obstacle by employing a combination of vector search and metadata filtering.

In this approach, we utilize the vector representation to perform similarity searches, looking for images that are visually similar to the query (shoe images). However, to fulfill the additional requirement of being released after a specific date, we leverage the metadata filtering step. By applying the date filter to the list of image metadata, we can identify and retrieve only the shoe images that match the desired release date criteria. This way, we effectively solve the problem of finding shoe images released after a particular date using a combination of vector search and metadata filtering.

Let's assume we have a music library with each song represented by a feature vector (encoded by a deep learning model) and accompanied by metadata such as "artist," "genre," and "release date." We'll explore three strategies for combining vector-based search and metadata filtering in this music library:

* Pre-selection filtering: In this approach, we first apply metadata filtering to identify a subset of songs that meet certain criteria, such as a specific artist or genre. We then mark these songs as eligible for vector-based similarity search. During the vector search process, we only consider the eligible songs and return the most similar songs from that subset.

* Post-selection filtering: Here, we begin by conducting a vector-based similarity search on the entire music library, finding a subset of songs that are semantically relevant to the query song. Once we have this subset, we then apply metadata filtering to further refine the results and obtain the final list of songs that match both the similarity and metadata criteria.

* Simultaneous search with filtering: In this strategy, we combine the vector search and metadata filtering steps during the search process itself. As we perform the similarity search on the feature vectors, we immediately filter out any songs that do not meet the specified metadata criteria. The search process continues until we collect the required number of songs that satisfy both similarity and metadata conditions.


A self-querying retriever is a retriever that possesses the unique capability of querying itself. True to its name, this retriever can take any natural language query and use a query-constructing Language Model (LLM) chain to generate a structured query. It then applies this structured query to its underlying VectorStore. Consequently, the retriever can not only perform semantic similarity comparisons between the user's input query and the contents of stored documents but also extract filters from the user's query based on the metadata of these stored documents. Additionally, the retriever can execute these extracted filters to further refine the search results.



In this notebook we explain how to approach the retriever pattern of semantice search with metadata filtering leveraging self quering with ChromaDB. 

Chroma is the open-source embedding database. Chroma makes it easy to build LLM apps by making knowledge, facts, and skills pluggable for LLMs.

Chroma gives you the tools to:

* store embeddings and their metadata
* embed documents and queries
* search embeddings

Chroma prioritizes:

* simplicity and developer productivity
* analysis on top of search
* it also happens to be very quick



## Usecase
#### Dataset
To explain this architecture pattern we are using a small demo set of documents that contain summaries of movies. 

#### Persona
Let's assume a persona of a customer trying to search for movies based on different criterias.




## Implementation
In order to follow the RAG approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V1 available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents
- **Document Loader**: PDF Loader available through LangChain

  This is the loader that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: FAISS available through LangChain

  In this notebook we are using ChromaDB vector-store to store both the embeddings and the documents. 
- **Index**: VectorIndex

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

### Setup
To run this notebook you would need to install 2 more dependencies, Lark and ChromaDB

Then begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude to demonstrate the use case.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

Available models under Bedrock have the following IDs:
- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`

In [122]:
# %pip install ../dependencies/botocore-1.30.1-py3-none-any.whl ../dependencies/boto3-1.27.1-py3-none-any.whl  --force-reinstall

Processing /root/bedrock-workshop-main/dependencies/botocore-1.30.1-py3-none-any.whl
Processing /root/bedrock-workshop-main/dependencies/boto3-1.27.1-py3-none-any.whl
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df7728471/urllib3-1.26.16-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.16-py2.py3-none-any.whl.metadata (48 kB)
  Obtaining dependency information for s3transfer<0.7.0,>=0.6.0 from https://files.pythonhosted.org/packages/d9/17/a3b666f5ef9543cfd3c661d39d1e193abb9649d0cfbbfee3cf3b51d5af02/s3transfer-0.6.2-py3-none-any.whl.metadata
  Using cached s3transfer-0.6.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
Using cached s3transfer-0.6.2-py3-none-any.whl (79 kB)
Using cached urllib3-1.26.1

In [3]:
%pip install langchain==0.0.271 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
awscli 1.29.14 requires botocore==1.31.14, but you have botocore 1.31.21 which is incompatible.
botocore 1.31.21 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.0.4 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.2 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.0 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.0 which is incompatible.
notebook 6.5.5 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.3.0 which is incompatible.
notebook 6.5.5 

#### Install ChromaDB and Lark packages

The self-query retriever requires you to have  chromadb package and lark. Lark is a modern general-purpose parsing library for Python. With Lark, you can parse any context-free grammar, efficiently, with very little code.

In [8]:
%pip install chromadb==0.4.6 --force-reinstall # python client

  Obtaining dependency information for chromadb==0.4.6 from https://files.pythonhosted.org/packages/f1/c2/d882d8650ba7cdb23f69d3abfb6bf11104cd408ef4fbec74b0ec0f36b9ea/chromadb-0.4.6-py3-none-any.whl.metadata
  Using cached chromadb-0.4.6-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for requests>=2.28 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for pydantic<2.0,>=1.9 from https://files.pythonhosted.org/packages/bc/e0/0371e9b6c910afe502e5fe18cc94562bfd9399617c7b4f5b6e13c29115b3/pydantic-1.10.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pydantic-1.10.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (149 kB)
  Using cached chroma-hnswlib-0.7.2.tar.gz (31 kB)
  Installing build dependencies ... do

In [9]:
%pip install lark==1.1.7 --force-reinstall

  Obtaining dependency information for lark==1.1.7 from https://files.pythonhosted.org/packages/9a/16/54258d6b368db265bde53515fe7ccb7c261b4ff3591b6354531abfe922ef/lark-1.1.7-py3-none-any.whl.metadata
  Using cached lark-1.1.7-py3-none-any.whl.metadata (1.9 kB)
Using cached lark-1.1.7-py3-none-any.whl (108 kB)
  Attempting uninstall: lark
    Found existing installation: lark 1.1.7
    Uninstalling lark-1.1.7:
      Successfully uninstalled lark-1.1.7
Note: you may need to restart the kernel to use updated packages.


In [6]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

### Import all the necessary libraries and access bedrock

We will import all the necessary libraries and access bedrock

In [7]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'
boto3_bedrock = bedrock.get_bedrock_client(os.environ.get('BEDROCK_ASSUME_ROLE', None))

PydanticUserError: If you use `@root_validator` with pre=False (the default) you MUST specify `skip_on_failure=True`. Note that `@root_validator` is deprecated and should be replaced with `@model_validator`.

For further information visit https://errors.pydantic.dev/2.3/u/root-validator-pre-skip

### Setup langchain

We create an instance of the Bedrock classes for the LLM and the embedding models. At the time of writing, Bedrock supports one embedding model and therefore we do not need to specify any model id.

 The TokenCounterHandler callback function is a function you can utilize in your LLM objects and chains to generate reports on token count. It is supplied here as a utility class that will output the token counts at the end of your result chain, or can be attached to a LLM object and invoked manually.

In [ ]:
%pip install tiktoken==0.4.0 --force-reinstall

In [ ]:
from utils.TokenCounterHandler import TokenCounterHandler

token_counter = TokenCounterHandler() 

With your LLM ready to go, you'll create a prompt template to utilize context to answer a given question. Prompt formats will be different by model, so if you change your model you will also likely need to adjust your prompt.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Here is a set of context, contained in <context> tags:

<context>
{context}
</context>

Use the context to provide an answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{question}

Assistant:"""

claude_choice_select_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"])

In [ ]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':500},  callbacks=[token_counter])
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)

### Creating a Chroma Vectorstore

First we'll want to create a Chroma VectorStore and seed it with some data. We have assembled a compact demonstration set of documents, each comprising movie summaries.

In [ ]:
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.vectorstores import Chroma
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "rating": 9.9,
            "director": "Andrei Tarkovsky",
            "genre": "science fiction",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, bedrock_embeddings)

### Creating our self-querying retriever

Now, to create our retriever instance, we need to provide some initial information regarding the metadata fields supported by our documents and a brief description of the document contents.

In [145]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"

retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True,callbacks=[token_counter]
)

### Testing it out

In [146]:
# This example only specifies a relevant query
query="What are some movies about dinosaurs"
retriever.get_relevant_documents(query)


query='dinosaur' filter=None limit=None


[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}),
 Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}),
 Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}),
 Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993})]

In [147]:
token_counter.report()


Token Counts:
Total: 3972
Embedding: N/A
Prompt: 3418
Generation:554



In [148]:
# This example only specifies a filter
retriever.get_relevant_documents("I want to watch a movie rated higher than 8.5")

query='movie rated' filter=Comparison(comparator=<Comparator.GT: 'gt'>, attribute='rating', value=8.5) limit=None


[Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006})]

In [149]:
token_counter.report()


Token Counts:
Total: 4858
Embedding: N/A
Prompt: 4276
Generation:582



In [150]:
# This example specifies a query and a filter
retriever.get_relevant_documents("Has Greta Gerwig directed any movies about women")

query='woman' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Greta Gerwig') limit=None


[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019}),
 Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019}),
 Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019}),
 Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019})]

In [151]:
token_counter.report()


Token Counts:
Total: 5743
Embedding: N/A
Prompt: 5131
Generation:612



In [152]:
# This example specifies a composite filter
retriever.get_relevant_documents(
    "What's a highly rated (above 8.5) science fiction film?"
)

query='science fiction' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GT: 'gt'>, attribute='rating', value=8.5), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction')]) limit=None


[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'science fiction', 'rating': 9.9, 'year': 1979}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'science fiction', 'rating': 9.9, 'year': 1979}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'science fiction', 'rating': 9.9, 'year': 1979}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'science fiction', 'rating': 9.9, 'year': 1979})]

In [153]:
token_counter.report()


Token Counts:
Total: 6643
Embedding: N/A
Prompt: 5991
Generation:652



In [154]:
# This example specifies a query and composite filter
retriever.get_relevant_documents(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

query='toys animated' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GT: 'gt'>, attribute='year', value=1990), Comparison(comparator=<Comparator.LT: 'lt'>, attribute='year', value=2005)]) limit=None


[Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995})]

In [155]:
token_counter.report()


Token Counts:
Total: 7551
Embedding: N/A
Prompt: 6859
Generation:692



### Filter K

We can also use the self query retriever to specify k: the number of documents to fetch.

We can do this by passing enable_limit=True to the constructor.

In [156]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
)

In [157]:
# This example only specifies a relevant query
retriever.get_relevant_documents("what are two movies about dinosaurs")

query='dinosaurs' filter=None limit=2


[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}),
 Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993})]

In [158]:
token_counter.report()


Token Counts:
Total: 8637
Embedding: N/A
Prompt: 7915
Generation:722



## Conclusion
Congratulations on completing this moduel on retrieval augmented generation with self-querying meta data filtering! This is an important technique that combines the power of large language models with the precision of retrieval methods. By augmenting generation with relevant retrieved examples, the responses we recieved become more coherent, consistent and grounded. You should feel proud of learning this innovative approach. I'm sure the knowledge you've gained will be very useful for building creative and engaging language generation systems. Well done!

In the above implementation of RAG based Question Answering we have explored the following concepts and how to implement them using Amazon Bedrock and it's LangChain integration.

- Loading documents and generating embeddings to create a vector store
- Retrieving documents to the question
- Preparing a prompt which goes as input to the LLM
- Present an answer in a human friendly manner

### Take-aways
- Experiment with different Vector Stores
- Leverage various models available under Amazon Bedrock to see alternate outputs
- Explore options such as persistent storage of embeddings and document chunks
- Integration with enterprise data stores

# Thank You